In [2]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
from dotenv import load_dotenv
from scipy.fft import fft, fftfreq

import os
import glob
import pandas as pd
import json

from utils.db_tools import get_db, filter_df, make_animation, get_data, metrics_grid, plot_grid
from classify import *

In [44]:
model = "bruss"
run_id = "abd_test"
load_dotenv()
data_dir = os.getenv("DATA_DIR")
output_dir = os.getenv("OUT_DIR")
output_dir = os.path.join(output_dir, model, run_id)
os.makedirs(output_dir, exist_ok=True)
df0 = get_db(os.path.join(data_dir, model, run_id))
len(df0)

378

In [45]:
df = df0.copy()

In [ ]:
df =

In [46]:
for Du in [1, 2, 3]:
    Dv = 4 * Du
    print(f"Du = {Du}, Dv = {Dv}")
    df_filt = filter_df(df, Du=Du, Dv=Dv)
    df_filt = classify_trajectories(df_filt, 0.1, convergence_eps=0.01, deriv_eps=0.05)
    for A in df_filt.A.unique():
        df_A = filter_df(df_filt,A)
        for B in df_A.B.unique():
            print(f"{filter_df(df_A,A,B).iloc[0]['category']:3}", end=" ")
        print("")

Du = 1, Dv = 4
SS  SS  SS  SS  SS  SS  SS  
SS  SS  SS  OSC ND  ND  ND  
SS  SS  OSC ND  ND  ND  ND  
SS  SS  DS  ND  ND  ND  DS  
SS  SS  SS  OSC ND  ND  DS  
SS  SS  SS  SS  SS  DS  DS  
Du = 2, Dv = 8
SS  SS  SS  SS  SS  SS  SS  
SS  SS  SS  OSC ND  ND  ND  
SS  SS  SS  ND  ND  ND  ND  
SS  SS  DS  ND  ND  ND  ND  
SS  SS  SS  OSC ND  ND  DS  
SS  SS  SS  SS  SS  ND  DS  
Du = 3, Dv = 12
SS  SS  SS  SS  SS  SS  SS  
SS  SS  SS  SS  ND  ND  ND  
SS  SS  SS  ND  ND  ND  ND  
SS  SS  DS  ND  ND  ND  ND  
SS  SS  SS  ND  ND  ND  DS  
SS  SS  SS  SS  SS  DS  DS  


In [30]:
for A in df.A.unique():
    for B in filter_df(df,A).B.unique():
        print(f"{filter_df(df,A,B).iloc[0]['category']:3}", end=" ")
    print("")

SS  SS  SS  SS  SS  SS  SS  
SS  SS  SS  OSC ?   ?   ?   
SS  SS  OSC ?   ?   ?   ?   
SS  SS  DS  ?   ?   ?   DS  
SS  SS  SS  OSC ?   ?   DS  
SS  SS  SS  SS  SS  DS  DS  


In [23]:
metrics_grid(df, 60, sigdigits=3, var1="A", var2="B", metric="dx", filename="dx-Du_1-Dv_4.png");

In [13]:
df2 = filter_df(df, 5, 12.5, 1, 4)
make_animation(get_data(df2), "5-12-osc", "")

MovieWriter ffmpeg unavailable; using Pillow instead.
